# XG Boost Model for Attributing Terrorist Attack Responsibility

In [1]:
### --- Libraries

import pandas as pd 
import xgboost as xgb 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import os

In [2]:
current_directory = os.getcwd()
print(current_directory)

C:\Users\ThomasMorgan\Desktop\Research\R\Github\GTI_2024\03_scripts\01_cleaning\Machine_Learning


# Setting the Variables

In [3]:
# Categorical Variables
ML_CATEGORICAL = ["geocode", "the_west", "event_type", "is_claimed", "suicide", "targets", "weapons", 
"intensity", "conflict", "admin_ID"]

TRAINING_DATA = "../../../02_data/processed/ml_testtraining.csv"

# Hyperparameters
#  hyperparameters = {
   # 'n_estimators': [50, 100, 150],
   # 'max_depth': [3, 6, 9],
   # 'learning_rate': [0.01, 0.1, 0.3],
   # 'gamma': [0, 0.5, 1],
   # 'colsample_bytree': [0.5, 0.7, 1],
   # 'min_child_weight': [1, 3, 5]
# }


# Hyperparameters
hyperparameters = {
    'n_estimators': [100],
    'max_depth': [6],
    'learning_rate': [0.1],
    'gamma': [0.5],
    'colsample_bytree': [0.7],
    'min_child_weight': [3]
}

# Functions for Running the Model

In [4]:
### --- Functions

# Load and process datasets
def pf_LoadClean(filepath):
    """Load and preprocess the dataset."""
    # Load data
    df = pd.read_csv(filepath)

    # Remove groups with 5 or fewer instances
    group_counts = df['Group'].value_counts()
    df = df[df['Group'].isin(group_counts[group_counts > 5].index)]

    # Encode categorical variables
    df = pd.get_dummies(df, columns=ML_CATEGORICAL)

    # Label Encoding for the target variable
    label_encoder = LabelEncoder()
    df['Group'] = label_encoder.fit_transform(df['Group'])

    return df, label_encoder

# Train the model
def pf_TrainModel(df, target_variable, hyperparameters):
    X = df.drop(target_variable, axis=1)
    y = df[target_variable]

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

    # Upsampling
    ros = RandomOverSampler(random_state=123)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    # Initialize XGBoost classifier
    model = xgb.XGBClassifier()

    # Grid search for hyperparameter tuning
    clf = GridSearchCV(model, hyperparameters, scoring='accuracy', cv=5)
    clf.fit(X_train_resampled, y_train_resampled)

    return clf, X_test, y_test

# Fit Model to Test Set and Evaluation
def pf_EvaluateModel(clf, X_test, y_test, group_encoder):
    predictions = clf.predict(X_test)

    # Convert numeric predictions back to original labels
    original_labels = group_encoder.inverse_transform(predictions)

    accuracy = accuracy_score(y_test, predictions)
    print(f"Model Accuracy: {accuracy:.2f}")

    return accuracy, original_labels

# Plot of feature importance
def pf_Plot_FeatureImportance(clf):
    """Plot feature importance of the model."""
    xgb.plot_importance(clf.best_estimator_)
    plt.show()

# Run the Training Model

In [5]:
# Load and preprocess data
df_evaluate, group_encoder = pf_LoadClean(TRAINING_DATA)

In [6]:
# Train the model
clf, X_test, y_test = pf_TrainModel(df_evaluate, 'Group', hyperparameters)

KeyboardInterrupt: 

In [ ]:
# Evaluate the model and get original labels of predictions
accuracy, original_labels = pf_EvaluateModel(clf, X_test, y_test, group_encoder)

# Print original labels of predictions
print("Original Labels of Predictions:", original_labels)

In [ ]:
# Plot feature importance
pf_Plot_FeatureImportance(clf)